In [1]:
%load_ext autoreload
%autoreload 2
import pysftp
import os
import pandas_gbq
import pandas as pd
from modules.buckets import *
from modules.reproducibility import *
from modules.stacking_bluff_asd import *
from modules.sftp_ops import *
import logging

clear_logging_handlers()

#Configure loggging
logging.basicConfig(filename='logs/BigQuery.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New Big Query Logging Instance')

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable in order to interact, import the SFTP password from the same file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'powerschool-420113-db919282054b.json'
    
# ----------------------------------------------------------
#Need to make this portion to where it assesses all files in the dir recursively. 
def main(SFTP_folder_name):

    SFTP_folder_name  = initial_schema_check(SFTP_folder_name)
    print(SFTP_folder_name)

    instance = Create(
                project_id='powerschool-420113',
                location = 'us-central1',
                bucket=f'{SFTP_folder_name}bucket-iotaschools-1',
                local_dir = fr'S:\SFTP\{SFTP_folder_name}',
                db = SFTP_folder_name,
                append_or_replace='replace',
                )
    
    instance.process()# Pass SFTP files into Bucket & then into Big Query tables

#roughly 4 mins to stack and send to new dir
# directory_path_blf = r'S:\SFTP\powerschool_tpcsc'
# directory_path_asd = r'S:\SFTP\powerschool'
# output_directory = r'S:\SFTP\powerschool_combined'
# concat_files_from_directories(directory_path_blf, directory_path_asd, output_directory)

# main("powerschool_combined")
# main("EIS")
main("misc_imports")
logging.info('Process has reached the end\n\n')

#Fix indentation of the logs
#Implement new SFTP module

C:\Users\amy.hardy\AppData\Roaming\Python\Python312\site-packages\pysftp\__init__.py:61: UserWarning: Failed to load HostKeys from C:\Users\amy.hardy\.ssh\known_hosts.  You will need to explicitly load HostKeys (cnopts.hostkeys.load(filename)) or disableHostKey checking (cnopts.hostkeys = None).
  warnings.warn(wmsg, UserWarning)


misc_imports
us-central1


Bucket misc_importsbucket-iotaschools-1 created in us-central1 with STANDARD storage class.
Local File S:\SFTP\misc_imports\google-powerschool-students-passwords.csv was uploaded as a new file google-powerschool-students-passwords.csv in the bucket misc_importsbucket-iotaschools-1.
Local File S:\SFTP\misc_imports\google-students-passwords.csv was uploaded as a new file google-students-passwords.csv in the bucket misc_importsbucket-iotaschools-1.
Local File S:\SFTP\misc_imports\TN_options_report_2014.txt was uploaded as a new file TN_options_report_2014.txt in the bucket misc_importsbucket-iotaschools-1.
misc_importsbucket-iotaschools-1
Attempting to create table powerschool-420113.misc_imports.TN_options_report_2014 and send data for the first time
Attempting to create table powerschool-420113.misc_imports.google-powerschool-students-passwords and send data for the first time
Attempting to create table powerschool-420113.misc_imports.google-students-passwords

In [5]:
temp = pd.read_csv("S:\SFTP\misc_imports\google-powerschool-students-passwords.csv", header=None)
# Manually rename the columns to "student_number", "email", and "password"
temp.columns = ["student_number", "email", "password"]


# upload_to_bq_table occurs within process. If the table_name = google-powerschool-students-passwords.csv

#This stems back to read_file which is in reproducibility. 
#Catch that error. Give optional column names

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\amy.hardy\AppData\Local\Temp\2\ipykernel_20460\2731113439.py:1: SyntaxWarning: invalid escape sequence '\S'
  temp = pd.read_csv("S:\SFTP\misc_imports\google-powerschool-students-passwords.csv", header=None)
